# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

# Get N Time Bars 

In this notebook, you will learn to create N time bars from the current time. The N represents the lookback period. You already have learned that we are using three input features; candlestick bars, technical indicators, and time signature to construct a state. For creating candlestick bars and technical indicators, we required time bars of last N period. 

![title](https://d2a032ejo53cab.cloudfront.net/Glossary/90h7GDAv/Untitled-Diagram-1.png)


The steps used to create N time bars are:
1. [Read price data](#read)
2. [Get N 5 mins bars ](#5mins)
3. [Resample price data ](#resample)
4. [Get N time bars](#timebars)

<a id='read'></a> 
## Read Price data

In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import datetime

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# The data is stored in the directory 'data_modules'
path = '../data_modules/'

# Read the pickle file
bars5m = pd.read_pickle(path+ 'PriceData5m.bz2')
bars5m.head()

,open,high,low,close,volume
Time,,,,,
2010-01-04 09:35:00-05:00,91.711,91.809,91.703,91.760,4448908.0
2010-01-04 09:40:00-05:00,91.752,91.973,91.752,91.932,4380988.0
2010-01-04 09:45:00-05:00,91.940,92.022,91.928,92.005,2876633.0
2010-01-04 09:50:00-05:00,92.005,92.177,91.973,92.177,4357079.0
2010-01-04 09:55:00-05:00,92.168,92.177,92.038,92.079,2955068.0


<a id='5mins'></a> 
### Get N 5 minutes bars 

In this step, we will illustrate how to get 5 minutes bars based on the lookback period. First, we will reduce the size of the data by defining the width of the data to 9 days (arbitrarily chosen). This is done to reduce the computation power as whenever you create time bars for certain lookback, you need to pull data from the start date. Next, you define the lookback period and make the times bars based on the specified lookback period.

We are also making sure that the length of the bars equals to the lookback period. This is done using the `assert` keyword. 

The assert keyword lets you test if a condition in your code returns True, if not, the code will raise an exception. For example, if the length of your created bars is 70 and you passed the lookback of 100, then it will raise an exception.

In [2]:
def get_last_N_5m_bars(bars5m, curr_time, lkbk):
    '''This function gets the timebars for the 5m resolution based
    on the lookback we've specified.
    '''
    # Width of the data, arbitrarly chosen
    wdw5m = 9

    """---creating the time bars based on the lookback---"""
    # Reduce the size of the data
    curtail_data = bars5m[curr_time-timedelta(wdw5m):curr_time]
    # Create candlestick based on lookback window 
    last5m = curtail_data.iloc[-lkbk:]

    '''---Making sure that window lengths agree with lookback---'''
    try:
        assert(len(last5m) == lkbk)

    except Exception as e:
        print('****Window length too short****')

    return last5m

In [3]:
curr_time = datetime.datetime(2012, 5, 10, 11, 0, 0)
lkbk = 5
get_last_N_5m_bars(bars5m, curr_time, lkbk)

,open,high,low,close,volume
Time,,,,,
2012-05-10 10:40:00-04:00,116.254,116.339,116.237,116.314,1951331.0
2012-05-10 10:45:00-04:00,116.314,116.425,116.297,116.374,1917502.0
2012-05-10 10:50:00-04:00,116.365,116.374,116.237,116.289,1517253.0
2012-05-10 10:55:00-04:00,116.297,116.297,116.143,116.160,1760156.0
2012-05-10 11:00:00-04:00,116.152,116.169,115.990,116.160,3202995.0


In the above output, you can see, we have created 5 mins timebar for lookback of 5 period from the mentioned time period. You can change the `lkbk` and `curr_time` parameter and experiment with that. You can also try to use the large value of lookback to check when the code raise an exception.

<a id='resample'></a> 
## Resample price data

Since we will work with 5 min, 1 hour and 1 day data, we need to resmaple the 5 mins data to these granularity. We have already discussed the resample method in the "Initialise and Reset Game Class" notebook.

In [4]:
ohlcv_dict = {
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    }


bars1h = bars5m.resample('1H', closed='right', label='left').agg(ohlcv_dict).dropna()
bars1d = bars5m.resample('1D', closed='right', label='left').agg(ohlcv_dict).dropna()

<a id='timebars'></a> 
## Get N time bars

Similar to the above code, we first define the width(time interval, in days) of the 5 mins, 1 hour and 1 day data. Width for the 5 minutes is 9 days. 

For width of 1 hour time bars, we will multiply the lookback period with 15 in order to create a time interval(width) wider than lookback period and divide it by 24 as there are 24 hours in a day. Finally, we will apply `ceil` method on that. This is done to make sure that the width should be greater than lookback period and that's why we multiply lookback period with a arbitrarily chosen number(15) higher than lookback period.

For width of 1 day time bars, we will multiply the lookback period with 15(to have a wider time interval than lookback period) and apply `ceil` method on that. 

Syntax: 
```python
import numpy as np
np.ceil(x)
```
Returns: smallest integer not less than x.

Example: np.ceil(17.3) == 18

In [5]:
def get_last_N_timebars(bars5m, bars1h, bars1d, curr_time, lkbk):
    '''This function gets the timebars for the 5m, 1hr and 1d resolution based
    on the lookback we've specified.
    '''
   
    """ Width of the 5m, 1hr, and 1d"""
    wdw5m = 9
    wdw1h = np.ceil(lkbk*15/24.)
    wdw1d = np.ceil(lkbk*15)

    """---creating the timebars based on the lookback---"""
    last5m = bars5m[curr_time-timedelta(wdw5m):curr_time].iloc[-lkbk:]
    last1h = bars1h[curr_time-timedelta(wdw1h):curr_time].iloc[-lkbk:]
    last1d = bars1d[curr_time-timedelta(wdw1d):curr_time].iloc[-lkbk:]

    '''---Making sure that window lengths agree with lookback---'''
    try:
        assert(len(last5m) == lkbk)
        assert(len(last1h) == lkbk)
        assert(len(last1d) == lkbk)

    except Exception as e:
        print('****Window length too short****')

    return last5m, last1h, last1d

In [6]:
curr_time = datetime.datetime(2012, 5, 10, 11, 0, 0)
lkbk = 5
get_last_N_timebars(bars5m, bars1h, bars1d, curr_time, lkbk)

(                              open     high      low    close     volume
 Time                                                                    
 2012-05-10 10:40:00-04:00  116.254  116.339  116.237  116.314  1951331.0
 2012-05-10 10:45:00-04:00  116.314  116.425  116.297  116.374  1917502.0
 2012-05-10 10:50:00-04:00  116.365  116.374  116.237  116.289  1517253.0
 2012-05-10 10:55:00-04:00  116.297  116.297  116.143  116.160  1760156.0
 2012-05-10 11:00:00-04:00  116.152  116.169  115.990  116.160  3202995.0,
                               open     high      low    close      volume
 Time                                                                     
 2012-05-09 14:00:00-04:00  116.331  116.382  115.955  115.973  18745626.0
 2012-05-09 15:00:00-04:00  115.964  116.220  115.742  115.827  34107806.0
 2012-05-10 09:00:00-04:00  116.638  116.783  116.408  116.587  14498502.0
 2012-05-10 10:00:00-04:00  116.587  116.732  115.990  116.160  28210656.0
 2012-05-10 11:00:00-04:00  116

We have created 5 min, 1 hour, 1 day timebar for lookback of 5 periods from the mentioned time period.
In the upcoming notebook, you will learn to use these bars to create the input features for the state construction.<br><br>